# Natural Language Processing

## Importing the libraries

In [90]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

## Importing the dataset

In [91]:
dataset = pd.read_csv('train.csv')

In [92]:
dataset.head(10)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1
6,10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1
8,14,NaN,NaN,There's an emergency evacuation happening now ...,1
9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1


In [93]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [94]:
dataset.isnull().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

## Cleaning the texts

In [95]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus = []
for i in range(0, 7613):
  text = re.sub('[^a-zA-Z]', ' ', dataset['text'][i])
  specific_cell = dataset.loc[i, 'text']
  #print(specific_cell)
  text = text.lower()
  text = text.split()
  #print(text)
  ps = PorterStemmer()
  all_stopwords = stopwords.words('english')
  all_stopwords.remove('not')
  text = [ps.stem(word) for word in text if not word in set(all_stopwords)]
  #print (text)
  text = ' '.join(text)
  corpus.append(text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Creating the Bag of Words model

In [97]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 2000)
X = cv.fit_transform(corpus).toarray()
y = dataset.iloc[:, -1].values

In [98]:
print (X)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]]


In [99]:
print (y)

[1 1 1 ... 1 1 1]


## Splitting the dataset into the Training set and Test set

In [100]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [101]:
print (y_test)

[0 0 0 ... 1 0 1]


## Training the Naive Bayes model on the Training set

In [102]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

GaussianNB()

In [103]:
!pip install catboost

In [ ]:
from catboost import CatBoostClassifier
classifierCAT = CatBoostClassifier()
classifierCAT.fit(X_train, y_train)


In [105]:
from xgboost import XGBClassifier
classifierXG = XGBClassifier()
classifierXG.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

## Predicting the Test set results

In [106]:
y_pred = classifier.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[0 0]
 [0 0]
 [1 0]
 ...
 [1 1]
 [0 0]
 [1 1]]


In [107]:
y_predCAT = classifierCAT.predict(X_test)
print(np.concatenate((y_predCAT.reshape(len(y_predCAT),1), y_test.reshape(len(y_test),1)),1))

[[0 0]
 [0 0]
 [1 0]
 ...
 [1 1]
 [0 0]
 [0 1]]


In [108]:
y_predXG = classifierXG.predict(X_test)
print(np.concatenate((y_predXG.reshape(len(y_predXG),1), y_test.reshape(len(y_test),1)),1))

[[0 0]
 [0 0]
 [1 0]
 ...
 [1 1]
 [0 0]
 [0 1]]


## Making the Confusion Matrix

In [109]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy =accuracy_score(y_test, y_pred)
print(accuracy)
f1 = f1_score(y_test, y_pred)
print (f1)

[[752 134]
 [230 407]]
0.7609980302035456
0.6910016977928691


In [110]:
cmCAT = confusion_matrix(y_test, y_predCAT)
print(cmCAT)
accuracyCAT =accuracy_score(y_test, y_predCAT)
print(accuracyCAT)
f1CAT = f1_score(y_test, y_predCAT)
print (f1CAT)

[[812  74]
 [221 416]]
0.8063033486539725
0.738243123336291


In [111]:
cmXG = confusion_matrix(y_test, y_predXG)
print(cmXG)
accuracyXG =accuracy_score(y_test, y_predXG)
print(accuracyXG)
f1XG = f1_score(y_test, y_predXG)
print (f1XG)

[[806  80]
 [219 418]]
0.8036769533814839
0.7365638766519824


In [112]:
#PREDICTING THE TEST RESULT FROM NEW CSV

In [113]:
 testDataset = pd.read_csv('test.csv')

In [114]:
testDataset.isnull().sum()

id             0
keyword       26
location    1105
text           0
dtype: int64

In [117]:
testDataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        3263 non-null   int64 
 1   keyword   3237 non-null   object
 2   location  2158 non-null   object
 3   text      3263 non-null   object
dtypes: int64(1), object(3)
memory usage: 102.1+ KB


In [115]:
testDataset.head(10)

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
5,12,NaN,NaN,We're shaking...It's an earthquake
6,21,NaN,NaN,They'd probably still show more life than Arse...
7,22,NaN,NaN,Hey! How are you?
8,27,NaN,NaN,What a nice hat?
9,29,NaN,NaN,Fuck off!


In [118]:
corpus = []
for i in range(0, 3263):
  text = re.sub('[^a-zA-Z]', ' ', testDataset['text'][i])
  specific_cell = testDataset.loc[i, 'text']
  #print(specific_cell)
  text = text.lower()
  text = text.split()
  #print(text)
  ps = PorterStemmer()
  text = [ps.stem(word) for word in text if not word in set(all_stopwords)]
  #print (text)
  text = ' '.join(text)
  corpus.append(text)

In [119]:
X_test_ = cv.fit_transform(corpus).toarray()

In [120]:
y_predCAT_ = classifierCAT.predict(X_test_)

In [121]:
print(y_predCAT_)

[0 0 0 ... 0 0 0]


In [122]:
testDataset['target'] = y_predCAT_

# Create a new DataFrame with only 'PassengerId' and 'Survived'
new_df = testDataset[['id', 'target']].copy()

# Now 'new_df' is a new DataFrame with the columns 'PassengerId' and 'Survived'
print(new_df)
# Save the new DataFrame to an Excel file
new_df.to_excel('NLPKaggle.xlsx', index=False)

         id  target
0         0       0
1         2       0
2         3       0
3         9       0
4        11       0
...     ...     ...
3258  10861       0
3259  10865       0
3260  10868       0
3261  10874       0
3262  10875       0

[3263 rows x 2 columns]
